In [1]:
from Bio.PDB import *
import urllib.request
import numpy as np
import pandas as pd
from math import sqrt
import time
import configparser
import os
import pymysql

In [2]:
config = configparser.ConfigParser()
config.read("/home/yuan/Documents/config.ini")
config.sections()

['office_ssh', 'office_mysql']

In [3]:
modification_site = pd.read_csv("/home/yuan/Documents/pdb_chain_ksites.csv")

In [4]:
modification_site = modification_site.sample(n=2000,replace=False,random_state=1)

In [12]:
modification_site = modification_site.reset_index()

In [13]:
modification_site_dic = {}
for i in range(len(modification_site)):
    if modification_site.loc[i,"PDB"] not in modification_site_dic:
        modification_site_dic[modification_site.loc[i,"PDB"]] = {modification_site.loc[i,"CHAIN"]:[modification_site.loc[i,"Ksites"]]}
    elif modification_site.loc[i,"CHAIN"] not in modification_site_dic[modification_site.loc[i,"PDB"]]:
        modification_site_dic[modification_site.loc[i,"PDB"]][modification_site.loc[i,"CHAIN"]] = [modification_site.loc[i,"Ksites"]]
    else:
        modification_site_dic[modification_site.loc[i,"PDB"]][modification_site.loc[i,"CHAIN"]].append(modification_site.loc[i,"Ksites"])
for protein in modification_site_dic:
    for chain in modification_site_dic[protein]:
        modification_site_dic[protein][chain] = list(modification_site_dic[protein][chain])


In [14]:
modification_site_dic

{'5gaf': {'G': [29, 27], 'M': [70], 'D': [62]},
 '4v5b': {'AM': [127],
  'BH': [68],
  'AN': [35],
  'AH': [89],
  'CM': [86],
  'CH': [41],
  'C1': [32],
  'CR': [10]},
 '4uy8': {'F': [46], 'N': [78], 'C': [146]},
 '6enf': {'b': [127], 'h': [21], 'l': [42]},
 '4v6o': {'B4': [49, 9],
  'BW': [25],
  'BM': [59],
  'BS': [111],
  'BO': [86, 127],
  'BG': [47],
  'B2': [39],
  'AI': [93]},
 '5wfk': {'p': [13],
  'H': [89],
  'e': [13],
  'V': [34, 85],
  'Y': [54],
  'd': [32]},
 '6qul': {'D': [160]},
 '4v9o': {'GV': [85, 25],
  'EV': [73],
  'AV': [34],
  'ES': [42],
  'ER': [10],
  'EH': [8],
  'CS': [73],
  'CD': [190],
  'GM': [86]},
 '3gef': {'B': [457]},
 '5owi': {'A': [361, 48]},
 '4v6e': {'DJ': [2], 'BE': [132], 'DD': [62]},
 '3d7s': {'B': [6], 'C': [83]},
 '6q9a': {'V': [85],
  'L': [13],
  'j': [14],
  'S': [28],
  'E': [48],
  'G': [89],
  'g': [28]},
 '4v4v': {'BF': [57],
  'AB': [127],
  'AJ': [11],
  'BD': [14],
  'BM': [88],
  'AF': [56],
  'AD': [155],
  'BE': [43]},
 '5lz

In [15]:
unique_list = modification_site[["PDB","CHAIN"]].drop_duplicates()

In [16]:
unique_list = unique_list.reset_index(drop=True)

In [17]:
backbone = ["N","CA","C","O"]
aminoAcidCodes = ["ALA","ARG","ASN","ASP","CYS","GLN","GLY","GLU","HIS","ILE","LEU","LYS",
                 "MET","PHE","PRO","PYL","SER","SEC","THR","TRP","TYR","VAL"]

In [19]:
PDB_ID = '3tch'
PDB = PDBList()
retrieve_status = PDB.retrieve_pdb_file(pdb_code = PDB_ID, pdir = "/mnt/HD1/pdb_data", file_format="pdb")
p = PDBParser()
structure = p.get_structure("X","/mnt/HD1/pdb_data/"+"pdb"+PDB_ID+".ent")

Structure exists: '/mnt/HD1/pdb_data/pdb3tch.ent' 


In [21]:
unique_list

,PDB,CHAIN
0,5gaf,G
1,4v5b,AM
2,4uy8,F
3,6enf,b
4,4v6o,B4
5,5wfk,p
6,6qul,D
7,4v9o,GV
8,3gef,B
9,5owi,A


In [22]:
conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                       passwd=config["office_mysql"]["sql_password"],
                       db=config["office_mysql"]["sql_main_database"],port=3306)
for index,oneRow in unique_list.iterrows():
    try:
        pdbID = oneRow["PDB"]
        chainOrder = oneRow["CHAIN"]
        retrieve_status = PDB.retrieve_pdb_file(pdb_code = pdbID, pdir = "/mnt/HD1/pdb_data", file_format="pdb")
        if os.path.exists(retrieve_status):
            p = PDBParser()
            structure = p.get_structure("X","/mnt/HD1/pdb_data/"+"pdb"+PDB_ID+".ent")
            if chainOrder in [x._id for x in list(structure.get_chains())]:
                oneChain = pd.DataFrame(columns = ["Seq","Residue","Center","Direction","CA_position"])#center here means center of R group
                if structure.header["resolution"] <= 3.0:
                    if chainOrder in [x.id for x in list(structure[0].get_chains())]:  # Chain information not in pdb file
                        for residue in structure[0][chainOrder]:
                            if residue.get_resname() in aminoAcidCodes:  # Only treat common amino acid
                                if residue.get_id()[2] == " ":  # ignore insertions
                                    if len(list(residue.get_atoms())) > 3:
                                        if residue.get_resname() != "GLY":  # Glysine as a special case
                                            point = vectors.Vector([0, 0, 0])
                                            for atom in residue:
                                                if (atom.get_name() not in backbone):
                                                    point = point + atom.get_vector()
                                            center = point.__div__(len(residue) - 4)
                                            cToRGroup = residue["CA"].get_vector() - center
                                            oneChain.loc[len(oneChain)] = [residue.get_id()[1], residue.get_resname(),center, cToRGroup,residue["CA"].get_vector()]
                                        else:
                                            center = residue["CA"].get_vector()
                                            cToRGroup = center - (residue["C"].get_vector() + residue["N"].get_vector() + residue["O"].get_vector()).__div__(3)
                                            oneChain.loc[len(oneChain)] = [residue.get_id()[1], residue.get_resname(),center, cToRGroup,residue["CA"].get_vector()]
                distanceMatrix = pd.DataFrame(columns=list(oneChain.iloc[:, 0]), index=list(oneChain.iloc[:,0]))
                angleMatrix = pd.DataFrame(columns=list(oneChain.iloc[:, 0]), index=list(oneChain.iloc[:, 0]))
                numResidue = len(oneChain)
                print(numResidue)
                for row in range(0, numResidue):
                    if row % 100 == 0:
                        print(str(row) + "th row")
                    for column in range(0, numResidue):#not symetric anymore
                        coordinatesSubstraction = list(oneChain.loc[row, "Center"] - oneChain.loc[column, "Center"])
                        pairDistance = sqrt(sum(list(map(lambda x: x * x, coordinatesSubstraction))))
                        distanceMatrix.iloc[row, column] = pairDistance
                        v1 = oneChain.loc[row, "Direction"]
                        v2 = oneChain.loc[column,"Center"] - oneChain.loc[row,"CA_position"] 
                        if abs(pairDistance-0)>0.00001: #Exclue 0 distance, e.g. self to self
                            pairAngle = 180 * np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))) / np.pi

                        else:
                            pairAngle = 0
                        angleMatrix.iloc[row, column] = pairAngle
                        if not np.isnan(pairDistance):
                            query = '''INSERT INTO `Distance_angle_PTM_sample` (`pdbID`, `chain`,`ID_1`,`Res_1`,`ID_2`,`Res_2`,`Distance`,`Angle`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s);'''
                            value = (pdbID,chainOrder,oneChain.iloc[row,0],oneChain.iloc[row,1],oneChain.iloc[column,0],oneChain.iloc[column,1],float(pairDistance),float(pairAngle))
                            #print(value)
                            conn.cursor().execute(query, value)
                    conn.commit()
    except Exception as e:
        print(e)
        print(pdbID, "skipped")

conn.close()

Structure exists: '/mnt/HD1/pdb_data/pdb5gaf.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb4uy8.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb6qul.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb3gef.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb5owi.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb3d7s.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb4pr9.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb3pfl.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb1m63.ent' 
517
0th row
100th row

517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb3p4q.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb4g1n.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb3bjf.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb2h75.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb2lni.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb2ms7.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb5cbu.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb3pdb.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb2jk4.ent' 
517
0th row
100th row
200th r

517
0th row
100th row
200th row
300th row
400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb6b58.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb3l4g.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb2jhw.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb3j5l.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb3cau.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb4lh3.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb1pcq.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb3iss.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb3pd6.ent' 
517
0th row
100th row
200th row
300th row
400th row
50

Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb2byx.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb4g1n.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb4a2i.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb4qty.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb1raa.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb4r67.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb6fve.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb1rqu.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb1nbe.ent' 
517
0th row
100th row
200

517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb3ezq.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb2air.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb1o8c.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb5gag.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb4k9g.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb2iop.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb2yey.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb5aka.ent' 
Structure exists: '/mnt/HD1/pdb_data/pd

...

Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb4q5e.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb2wp9.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb2oow.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb2byy.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb6j99.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb6mo6.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb6gbz.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb1sx4.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb5gad.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb4yqf.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb5vwo.ent' 
517
0th row
10

400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb2nu6.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb5gah.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb2wp9.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb5uaj.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb5omw.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb2bz3.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb6gc8.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb2a6l.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb1nqt.ent' 
Struc

517
0th row
100th row
200th row
300th row
400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb2bw0.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb5m36.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb4x43.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb1rah.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb3mpu.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb4kwf.ent' 
Desired structure

200th row
300th row
400th row
500th row
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb6gjh.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb1rdd.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb5dff.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb2wz0.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb2jhv.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb6n13.ent' 
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb2gg7.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb1qm5.ent' 
Des

Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb4j5b.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Structure exists: '/mnt/HD1/pdb_data/pdb6gc7.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb1szx.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb5ur9.ent' 
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb3f9k.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb2h73.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb4z1u.ent' 
Structure exists: '/mnt/HD1/pdb_data/pdb1nyl.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th row
Desired structure doesn't exists
Desired structure doesn't exists
Structure exists: '/mnt/HD1/pdb_data/pdb5aqi.ent' 
517
0th row
100th row
200th row
300th row
400th row
500th 

In [26]:
%%time
conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                       passwd=config["office_mysql"]["sql_password"],
                       db=config["office_mysql"]["sql_main_database"],port=3306)
try:
    with conn as cursor: #auto commit; no close() called
        with cursor: # close() called here
            sql_select = "Select * FROM pdbdb.Distance_angle_PTM_sample da "
            sql_where = "WHERE da.pdbID = \"1nyl\" and da.chain = \"A\" and da.ID_1=27 ;"    
            sql = sql_select+sql_where
            data = pd.read_sql_query(sql, conn)



 
except Exception as e: # catch exceptions
    print("~~~~~~~~~~~~~~")
    print(e)
finally:
    if conn:
        conn.close()

CPU times: user 9.29 ms, sys: 0 ns, total: 9.29 ms
Wall time: 9.29 ms


In [27]:
%%time
data

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 2.86 µs


,ID,pdbID,chain,ID_1,Res_1,ID_2,Res_2,Distance,Angle
0,73504476,1nyl,A,27,ALA,27,ALA,0.000000,0.000000
1,73504477,1nyl,A,27,ALA,28,ASP,6.046836,44.322787
2,73504478,1nyl,A,27,ALA,29,VAL,5.833208,125.540528
3,73504479,1nyl,A,27,ALA,30,PRO,8.773294,103.794832
4,73504480,1nyl,A,27,ALA,31,ALA,13.387449,94.654994
5,73504481,1nyl,A,27,ALA,32,GLY,14.014413,113.107648
6,73504482,1nyl,A,27,ALA,33,VAL,10.285939,127.268325
7,73504483,1nyl,A,27,ALA,34,THR,14.686654,139.301714
8,73504484,1nyl,A,27,ALA,35,LEU,8.364347,149.609354
9,73504485,1nyl,A,27,ALA,36,ALA,12.995875,170.161078
